In [1]:
import os
import tifffile as tiff
import skimage.io as io
from skimage.filters import threshold_otsu
import numpy as np
from os.path import join
from glob import glob
import sklearn.model_selection as skms
from shutil import copy2

In [8]:
# Define paths
DATA_PATH = '/dtu/3d-imaging-center/courses/02510/groups/group_Kakapo/Project/Manual_label/labels/test'
#TRAIN_IMAGE_DIR = os.path.join(DATA_PATH, 'train')

In [9]:
# Function to load and segment an image, then save the segmented version
def segment_and_save_image(image_path, save_dir):
    img = tiff.imread(image_path)
    otsu = threshold_otsu(img)
    segmented_img = img > otsu
    save_path = os.path.join(save_dir, os.path.basename(image_path).replace('.tif', '_segmented.tif'))
    io.imsave(save_path, segmented_img.astype(np.uint8) * 255)
    print(f"Segmented image saved to {save_path}")

# Make sure the output directory exists
output_dir = '/dtu/3d-imaging-center/courses/02510/groups/group_Kakapo/Project/Manual_label/labels/'
os.makedirs(output_dir, exist_ok=True)

# Process each image in the directory
for filename in os.listdir(DATA_PATH):
    if filename.endswith('.tif'):
        file_path = os.path.join(DATA_PATH, filename)
        segment_and_save_image(file_path, output_dir)

Segmented image saved to /dtu/3d-imaging-center/courses/02510/groups/group_Kakapo/Project/Manual_label/labels/bcrick_2_033_label_segmented.tif
Segmented image saved to /dtu/3d-imaging-center/courses/02510/groups/group_Kakapo/Project/Manual_label/labels/bcrick_2_031_label_segmented.tif


/tmp/ipykernel_18012/1840929632.py:7: UserWarning: /dtu/3d-imaging-center/courses/02510/groups/group_Kakapo/Project/Manual_label/labels/bcrick_2_033_label_segmented.tif is a low contrast image
  io.imsave(save_path, segmented_img.astype(np.uint8) * 255)


In [18]:
# Load data
DATA_PATH  = '/dtu/3d-imaging-center/courses/02510/groups/group_Kakapo/Project/'
IMAGE_DIR = 'bug_256/AC/'
LABEL_DIR = 'bug_256/'
#-----Old testing images-----
# TRAIN_IMAGE = 'train/bcrick_10_000.tif'
# TRAIN_LABEL = 'train/bcrick_10_000_annotated.tif'
# VAL_IMAGE = 'val/bcrick_10_001.tif'
# VAL_LABEL = 'val/bcrick_10_001_annotated.tif'
for split in ['train', 'val', 'test']:
    os.makedirs(join(DATA_PATH, IMAGE_DIR, split), exist_ok=True)
    os.makedirs(join(DATA_PATH, LABEL_DIR, split), exist_ok=True)

# Load all file names
image_files = sorted(glob(join(DATA_PATH, IMAGE_DIR, '*.tif')))
label_files = sorted(glob(join(DATA_PATH, LABEL_DIR, '*.tif')))

# Split data into train, val, test
train_images, test_images, train_labels, test_labels = skms.train_test_split(
    image_files, label_files, test_size=0.3, random_state=42)

# Further split the test set into validation and actual test sets
val_images, test_images, val_labels, test_labels = skms.train_test_split(
    test_images, test_labels, test_size=0.5, random_state=42)

#Function to copy files to their new location----inabled
def copy_files(files, labels, split):
    for img, lbl in zip(files, labels):
        # Copy images
        dest_img = join(DATA_PATH, IMAGE_DIR, split, os.path.basename(img))
        copy2(img, dest_img)
        # Copy labels
        dest_lbl = join(DATA_PATH, LABEL_DIR, split, os.path.basename(lbl))
        copy2(lbl, dest_lbl)

# Copy files to their respective directories
copy_files(train_images, train_labels, 'train')
copy_files(val_images, val_labels, 'val')
copy_files(test_images, test_labels, 'test')

print("Data successfully split and copied to respective folders.")

Data successfully split and copied to respective folders.
